In [ ]:
# Momentum Strategy: Long Top 10%, Short Bottom 10% of S&P 500
# Timeframe: Past 5 years, monthly rebalancing

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set plotting style
plt.style.use("seaborn-v0_8")
sns.set()
pd.options.mode.chained_assignment = None

# Parameters
start_date = "2019-06-01"
end_date = "2024-06-01"
lookback_days = 21  # ~1 trading month
percentile = 0.1  # Top/bottom 10% of returns

# Step 1: Load S&P 500 tickers from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(sp500_url)
tickers = table[0]['Symbol'].tolist()
tickers = [ticker.replace('.', '-') for ticker in tickers]  # BRK.B → BRK-B

# Step 2: Safely download price data
price_data = pd.DataFrame()
valid_tickers = []

print("Downloading data for S&P 500 tickers...")
for ticker in tqdm(tickers):
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)["Adj Close"]
        if not df.empty and not df.isna().all():
            price_data[ticker] = df
            valid_tickers.append(ticker)
    except Exception as e:
        print(f"Skipped {ticker}: {e}")

# Step 3: Calculate monthly returns
monthly_prices = price_data.resample("ME").last()
monthly_returns = monthly_prices.pct_change()

# Step 4: Momentum signal: previous month's return
signal = monthly_returns.shift(1)

# Step 5: Backtest long-short strategy
long_short_returns = []

for date in signal.index[1:]:
    ranked = signal.loc[date].dropna()
    if len(ranked) < 50:
        long_short_returns.append(0)
        continue

    top = ranked[ranked >= ranked.quantile(1 - percentile)]
    bottom = ranked[ranked <= ranked.quantile(percentile)]

    next_returns = monthly_returns.loc[date]

    long_return = next_returns[top.index].mean()
    short_return = next_returns[bottom.index].mean()

    portfolio_return = long_return - short_return
    long_short_returns.append(portfolio_return)

# Step 6: Create performance DataFrame
perf = pd.DataFrame({
    "Date": signal.index[1:],
    "Long-Short Return": long_short_returns
})
perf.set_index("Date", inplace=True)
perf["Cumulative Return"] = (1 + perf["Long-Short Return"]).cumprod()

# Step 7: Plot cumulative returns
plt.figure(figsize=(12, 6))
plt.plot(perf.index, perf["Cumulative Return"], label="Long Top 10% / Short Bottom 10%")
plt.title("Momentum Strategy Performance (5-Year Backtest)")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 8: Performance metrics
total_return = perf["Cumulative Return"].iloc[-1] - 1
annualized_return = perf["Cumulative Return"].iloc[-1]**(1/5) - 1
volatility = np.std(perf["Long-Short Return"]) * np.sqrt(12)
sharpe = annualized_return / volatility

print("\n📊 Strategy Performance Metrics:")
print(f"Total Return:         {total_return:.2%}")
print(f"Annualized Return:    {annualized_return:.2%}")
print(f"Annualized Volatility:{volatility:.2%}")
print(f"Sharpe Ratio:         {sharpe:.2f}")


  0%|                                                                                          | 0/503 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True



1 Failed download:
['MMM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
  0%|▏                                                                                 | 1/503 [00:03<26:50,  3.21s/it]
1 Failed download:
['AOS']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
  0%|▎                                                                                 | 2/503 [00:03<12:14,  1.47s/it]
1 Failed download:
['ABT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
  1%|▍                                                                                 | 3/503 [00:04<10:35,  1.27s/it]
1 Failed download:
['ABBV']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
  1%|▋                                                                                 | 4/503 [00:05<09:42,  1.17s/it]
1 Failed download:
['ACN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
  1%|▊            